In [1]:
client_id = 'hwEnzlSdGXZTyeubesmy'
client_secret = 'WkpCvqbw6u'

In [2]:
import os
import sys
import urllib.request
import json
import pandas as pd
import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re

In [16]:
# query	String	Y	검색어. UTF-8로 인코딩되어야 합니다.
# display	Integer	N	한 번에 표시할 검색 결과 개수(기본값: 10, 최댓값: 100)
# start	Integer	N	검색 시작 위치(기본값: 1, 최댓값: 1000)
# sort	String	N	검색 결과 정렬 방법
# - sim: 정확도순으로 내림차순 정렬(기본값)
# - date: 날짜순으로 내림차순 정렬

def crawl_news(query):
    news_list = []
    for step in range(1, 1100, 100):
        if step == 1001:
            step = 1000

        params = dict(
            query=urllib.parse.quote('책상'),
            display=urllib.parse.quote('100'),
            start=urllib.parse.quote(str(step)),
            sort=urllib.parse.quote('date'),
        )

        url = f"https://openapi.naver.com/v1/search/news?query={params['query']}&display={params['display']}&start={params['start']}&sort={params['sort']}"

        request = urllib.request.Request(url)
        request.add_header('X-Naver-Client-Id',client_id)
        request.add_header('X-Naver-Client-Secret',client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if(rescode==200):
            response_body = response.read()
            result = json.loads(response_body.decode('utf-8'))['items']
            news_list.extend(result)
        else:
            print('Error Code:' + rescode)
        # print(step)
        
    news_df = pd.DataFrame(news_list).drop_duplicates()
    news_df['checker'] = news_df.link.str.find('https://n.news.naver.com/')
    news_df = news_df[news_df['checker'] > -1]
    news_df.reset_index(inplace=True)
            
    return news_df

In [17]:
query = '인공지능'

In [18]:
news_df = crawl_news(query)

In [6]:
# for idx, row in news_df.iterrows():
#     print(row.link)
#     if idx == 50:
#         break

In [14]:
articles = []

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'cookie': 'nid_inf=1417715440; NID_AUT=jB6JKkl0bQJkzabA2sqcsZPcc58lN3Nk/oKjGyVYl1n8PedB7ujlxHdECKcm9kS7; NID_JKL=CdIfrElqYRkes7uT/S10prytLgVd7C1HBLS0nz55Sys=; VISIT_LOG_CLEAN=1; NFS=2; NID_SES=AAABkxQsSdlab5VkBVUMJHd12mikXaT/HRo/rtzzaxgjXmXJLfOFG25S5TiaZsFNk5bLeRF4Ej3UoZTevQC7Jd7fVFOAFJtSFoNgapi8qqUTVHDqzpWo4zUjmx/LD5cdsWbvaL/cByCai8XYLRLdB0o6W7sQutPmRMFUblZMKU/CdpBh8aziLA/i9wHRgkCr4zpYxc/YROLyZLZ7QmphI5qakKYHN4OxNKYy/Dp+PvlCsWk6azoXw7AxABwR/NdLMwDxNgZ6yN0iUJDkWFFPEN0yBGCm5VaOa1oF4tU0GkLMQsbyMb/+mOw6aBjxQNSgS5RDsn3Vs6H9l1qczNtasU4OpdonIK0EzoLYyQEgLkpXeLTnzkdaJnqR7EyjKjl8hH/EsaVdnRaAEs0afY3VK8f+pCn0z/5n9xpiCd0UVrpoI3nwF1No4x/mEk71EW2ghH90hyg/9F0KHAHLneYZ6lCvcdhE/ENHrmc1nP0R0sUnOltV0XIyf0fEua7rvSqv0P0iuE0xD2+WUqh//96WFUP+XyeQxvn16XPBT8PSfE/nU/FQ; nx_ssl=2; page_uid=h1WRNsp0YiRssKvZbrNssssssMh-389234; BMR='
}
# 네이버뉴스 일반: newsct_article 연예: #articeBody
for idx, row in news_df.iterrows():
    # row.title, row.originallink, row.link, row.description, row.pubDate,
    print(idx)
    URL = str(row.link)
    r = requests.get(URL, headers=headers)
    try:
        soup = BeautifulSoup(r.content, 'html5lib')
        
        article = soup.select('#newsct_article')[0].text
        cleansed_article = re.sub(r'[\n\t^/$]', '', article)
        articles.append([row.title, row.originallink, row.link, row.description, row.pubDate, cleansed_article])
    except Exception as e:
        print(e)
        soup = BeautifulSoup(r.content, 'html5lib')
        
        article = soup.select('#articeBody')[0].text
        cleansed_article = re.sub(r'[\n\t^/$]', '', article)
        articles.append([row.title, row.originallink, row.link, row.description, row.pubDate, cleansed_article])
    
    if idx == 10:
        break

0
1
2
3
4
list index out of range
5
6
list index out of range
7
list index out of range
8
list index out of range
9
list index out of range
10
list index out of range


In [15]:
for row in articles:
    print(row[5])
    print()

업사이클링 활용 도서관 설립, 탄소저감 위한 도심 숲 조성 등ESG경영(환경·사회·지배구조)이 글로벌 화두로 주목 받으며 기업 생존에 필수 요소로 자리 잡았다.롯데홈쇼핑(대표 이완신) 역시 기존 CSR, 준법경영 중심에서 친환경 활동을 강화한 ESG 경영을 전개하고 있다. 지난해 8월 ESG경영을 선포한 이후 친환경 도서관 조성, 도심 숲 조성 등 친환경 경영을 통한 사회적 책임 실천에 앞장서고 있다.■폐현수막 활용한 시민들의 힐링공간, 남산도서관에 친환경 독서공간 만들다롯데홈쇼핑은 2013년부터 구세군 자선냄비와 함께 문화 소외지역 아동들을 위해 친환경 학습공간 ‘작은도서관’을 운영하고 있다. 올해 1월 경북 구미에 의류 폐기물을 재가공해 제작한 가구로 ‘작은도서관’을 건립한 것을 시작으로, 업사이클링을 활용한 도서관 구축을 추진하고 있다. 지난 20일에는 작은도서관 사업의 일환으로 서울 남산도서관 옥외공간에 업사이클링 자재를 활용한 친환경 독서공간 ‘남산하늘뜰’을 조성하고 개관식을 진행했다.롯데홈쇼핑, 남산도서관에 친환경 독서공간 조성이번 사업은 지난 5월 롯데홈쇼핑이 서울특별시, 서울특별시 교육청, 환경공단과 체결한 ‘자원순환 및 ESG경영 실천을 위한 업무협약’의 첫 시범사업으로 기획됐다. 6.1 지방 선거에서 사용 후 폐기되는 현수막의 친환경적인 활용방법을 모색하다 서울 최초의 공립 공공도서관인 남산도서관의 개관 100주년을 기념해 친환경 독서공간을 조성하기로 결정했다.이에 남산도서관 옥외공간에 약 6개월 간의 작업 과정을 거쳐 업사이클링 자재를 활용한 친환경 독서공간 ‘남산하늘뜰’을 조성했다. 남산하늘뜰은 숲 속의 정자를 닮은 공간으로 디자인됐다. 모여서 책을 읽을 수 있는 정자를 닮은 공간, 잔디밭처럼 앉아서 쉴 수 있는 휴식공간, 그리고 자연을 느끼며 누구에게도 방해 받지 않고 집중할 수 있는 공간으로 구성됐다. 폐현수막 2천540장, 폐의류 2만2천860벌을 업사이클링 섬유패널 4.25톤으로 조형물, 벤치, 평상, 의자 등을 제작해 약 1.3톤

In [73]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'cookie': 'nid_inf=1417715440; NID_AUT=jB6JKkl0bQJkzabA2sqcsZPcc58lN3Nk/oKjGyVYl1n8PedB7ujlxHdECKcm9kS7; NID_JKL=CdIfrElqYRkes7uT/S10prytLgVd7C1HBLS0nz55Sys=; VISIT_LOG_CLEAN=1; NFS=2; NID_SES=AAABkxQsSdlab5VkBVUMJHd12mikXaT/HRo/rtzzaxgjXmXJLfOFG25S5TiaZsFNk5bLeRF4Ej3UoZTevQC7Jd7fVFOAFJtSFoNgapi8qqUTVHDqzpWo4zUjmx/LD5cdsWbvaL/cByCai8XYLRLdB0o6W7sQutPmRMFUblZMKU/CdpBh8aziLA/i9wHRgkCr4zpYxc/YROLyZLZ7QmphI5qakKYHN4OxNKYy/Dp+PvlCsWk6azoXw7AxABwR/NdLMwDxNgZ6yN0iUJDkWFFPEN0yBGCm5VaOa1oF4tU0GkLMQsbyMb/+mOw6aBjxQNSgS5RDsn3Vs6H9l1qczNtasU4OpdonIK0EzoLYyQEgLkpXeLTnzkdaJnqR7EyjKjl8hH/EsaVdnRaAEs0afY3VK8f+pCn0z/5n9xpiCd0UVrpoI3nwF1No4x/mEk71EW2ghH90hyg/9F0KHAHLneYZ6lCvcdhE/ENHrmc1nP0R0sUnOltV0XIyf0fEua7rvSqv0P0iuE0xD2+WUqh//96WFUP+XyeQxvn16XPBT8PSfE/nU/FQ; nx_ssl=2; page_uid=h1WRNsp0YiRssKvZbrNssssssMh-389234; BMR='
}

In [119]:
URL = "https://n.news.naver.com/mnews/article/076/0003932507?sid=106"
r = requests.get(URL, headers=headers)

soup = BeautifulSoup(r.content, 'html5lib')
print(soup.text)
# article = soup.select('.newsct_article')[0].text
# # article = soup.select('#newsct_article')[0].text
# cleansed_article = re.sub(r'[\n\t]', '', article)
# print(soup.prettify())

2022-10-28 14:28:51 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): n.news.naver.com:443
2022-10-28 14:28:51 [urllib3.connectionpool] DEBUG: https://n.news.naver.com:443 "GET /mnews/article/076/0003932507?sid=106 HTTP/1.1" 302 77
2022-10-28 14:28:51 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): entertain.naver.com:443
2022-10-28 14:28:51 [urllib3.connectionpool] DEBUG: https://entertain.naver.com:443 "GET /read?oid=076&aid=0003932507 HTTP/1.1" 200 None



	"남자 등쳐먹고 살아"…초3 욕쟁이 아들, 선생님한테도 막무가내 욕('금쪽') :: 네이버 TV연예
	
	


var doc = document.documentElement;
doc.setAttribute('data-useragent', navigator.userAgent);






// document.domain = 'naver.com';


(function () {
	var enterPcCookie = jindo.$Cookie().get("enter_pc");
	var _agent = jindo.$Agent().navigator();
	var isMobile = _agent.mobile || _agent.msafari;
	if (window.location.href.indexOf("viewType=pc") > 0 && (enterPcCookie == "false" || !enterPcCookie)) {
		jindo.$Cookie().set("enter_pc", "true", 0, "entertain.naver.com");
	} else if (isMobile && enterPcCookie != "true") {
		if ((window.location.href.indexOf("/photo/issue/") > 0) && (window.location.hash.indexOf("cid") > 0)) {
			var hashValue = window.location.hash;
			var obj = jindo.$S(hashValue).parseString()
			if (obj['#cid'] != '' && obj['iid']) {
				window.location.href = "https://m.entertain.naver.com/entertain?id=" + obj['#cid'] + "&imgId=" + obj['iid']
					+ "&host=https://m.entertain.naver.com" + "&listUrl=https://m.en

In [115]:
cleansed_article

"현대백화점 신촌점 유플렉스 지하 2층에 마련된 팝업스토어 도구리 오피스엔씨소프트는 캐릭터 브랜드 '도구리(DOGURI)'의 첫 번째 팝업스토어 '도구리 오피스'를 현대백화점 신촌점 유플렉스에 오픈했다고 28일 밝혔다.도구리 오피스는 도구리가 근무하는 '644컴퍼니' 사무실을 콘셉트로 구현했다. 방문객은 회의실, 도구리 책상, 탕비실, 비품실 등 다양한 참여형 공간 및 이벤트를 통해 캐릭터의 세계관을 직접 체험할 수 있다.팝업스토어 내 '면접존'에서는 도구리의 동료 사원을 뽑는 '막내사원 공개 채용 면접 이벤트'를 실시한다. 사전 예약에 참여한 방문객은 현장에서 면접을 체험하고 기념으로 커스텀 사원증과 목걸이를 받을 수 있다.방문객은 팝업스토어 현장에서 처음 공개되는 신규 굿즈도 만나볼 수 있다. 도구리 맨투맨, 도구리 2023 탁상달력, 도구리 노트북 파우치, 도구리 미니 피규어 등 다양한 제품을 오프라인에서 구매할 수 있다.도구리 팝업스토어는 10월 27일부터 11월 8일까지 현대백화점 신촌점 유플렉스 지하 2층에서 운영한다."

In [77]:
lst.append(cleansed_article)

In [78]:
lst

["현대백화점 신촌점 유플렉스 지하 2층에 마련된 팝업스토어 도구리 오피스엔씨소프트는 캐릭터 브랜드 '도구리(DOGURI)'의 첫 번째 팝업스토어 '도구리 오피스'를 현대백화점 신촌점 유플렉스에 오픈했다고 28일 밝혔다.도구리 오피스는 도구리가 근무하는 '644컴퍼니' 사무실을 콘셉트로 구현했다. 방문객은 회의실, 도구리 책상, 탕비실, 비품실 등 다양한 참여형 공간 및 이벤트를 통해 캐릭터의 세계관을 직접 체험할 수 있다.팝업스토어 내 '면접존'에서는 도구리의 동료 사원을 뽑는 '막내사원 공개 채용 면접 이벤트'를 실시한다. 사전 예약에 참여한 방문객은 현장에서 면접을 체험하고 기념으로 커스텀 사원증과 목걸이를 받을 수 있다.방문객은 팝업스토어 현장에서 처음 공개되는 신규 굿즈도 만나볼 수 있다. 도구리 맨투맨, 도구리 2023 탁상달력, 도구리 노트북 파우치, 도구리 미니 피규어 등 다양한 제품을 오프라인에서 구매할 수 있다.도구리 팝업스토어는 10월 27일부터 11월 8일까지 현대백화점 신촌점 유플렉스 지하 2층에서 운영한다."]

In [51]:

re.sub(r'[\n\t]', '', article)


"현대백화점 신촌점 유플렉스 지하 2층에 마련된 팝업스토어 도구리 오피스엔씨소프트는 캐릭터 브랜드 '도구리(DOGURI)'의 첫 번째 팝업스토어 '도구리 오피스'를 현대백화점 신촌점 유플렉스에 오픈했다고 28일 밝혔다.도구리 오피스는 도구리가 근무하는 '644컴퍼니' 사무실을 콘셉트로 구현했다. 방문객은 회의실, 도구리 책상, 탕비실, 비품실 등 다양한 참여형 공간 및 이벤트를 통해 캐릭터의 세계관을 직접 체험할 수 있다.팝업스토어 내 '면접존'에서는 도구리의 동료 사원을 뽑는 '막내사원 공개 채용 면접 이벤트'를 실시한다. 사전 예약에 참여한 방문객은 현장에서 면접을 체험하고 기념으로 커스텀 사원증과 목걸이를 받을 수 있다.방문객은 팝업스토어 현장에서 처음 공개되는 신규 굿즈도 만나볼 수 있다. 도구리 맨투맨, 도구리 2023 탁상달력, 도구리 노트북 파우치, 도구리 미니 피규어 등 다양한 제품을 오프라인에서 구매할 수 있다.도구리 팝업스토어는 10월 27일부터 11월 8일까지 현대백화점 신촌점 유플렉스 지하 2층에서 운영한다."

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

bodies = []

class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            # "https://n.news.naver.com/mnews/article/030/0003054809?sid=105",
            # 'https://n.news.naver.com/article/030/0003054809?sid=105',
            'https://n.news.naver.com/article/030/0003054809',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        
        print('*'*100)
        print(response.body)
        
        # page = response.url.split("/")[-2]
        # filename = f'quotes-{page}.html'
        # with open(filename, 'wb') as f:
        #     f.write(response.body)
        # self.log(f'Saved file {filename}')
        
process = CrawlerProcess(settings={
    "FEEDS": {
        "items.json": {"format": "json"},
    },
})

process.crawl(QuotesSpider)
process.start() # the script will block here until the crawling is finished

2022-10-28 13:27:57 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-10-28 13:27:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.10.6 (main, Oct  7 2022, 20:19:58) [GCC 11.2.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 35.0.0, Platform Linux-4.15.0-143-generic-x86_64-with-glibc2.27
2022-10-28 13:27:57 [scrapy.crawler] INFO: Overridden settings:
{}
2022-10-28 13:27:57 [py.warnings] WARNING: /home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See th

In [17]:
# -*- coding: utf-8 -*-
import scrapy
#import logging

class AaidSpider(scrapy.Spider):
    name = 'aaid'

    def parse(self, response):
        url = "https://n.news.naver.com/mnews/article/030/0003054809?sid=105"

        # Set the headers here. 
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
            'cookie': 'nid_inf=1417715440; NID_AUT=jB6JKkl0bQJkzabA2sqcsZPcc58lN3Nk/oKjGyVYl1n8PedB7ujlxHdECKcm9kS7; NID_JKL=CdIfrElqYRkes7uT/S10prytLgVd7C1HBLS0nz55Sys=; VISIT_LOG_CLEAN=1; NFS=2; NID_SES=AAABkxQsSdlab5VkBVUMJHd12mikXaT/HRo/rtzzaxgjXmXJLfOFG25S5TiaZsFNk5bLeRF4Ej3UoZTevQC7Jd7fVFOAFJtSFoNgapi8qqUTVHDqzpWo4zUjmx/LD5cdsWbvaL/cByCai8XYLRLdB0o6W7sQutPmRMFUblZMKU/CdpBh8aziLA/i9wHRgkCr4zpYxc/YROLyZLZ7QmphI5qakKYHN4OxNKYy/Dp+PvlCsWk6azoXw7AxABwR/NdLMwDxNgZ6yN0iUJDkWFFPEN0yBGCm5VaOa1oF4tU0GkLMQsbyMb/+mOw6aBjxQNSgS5RDsn3Vs6H9l1qczNtasU4OpdonIK0EzoLYyQEgLkpXeLTnzkdaJnqR7EyjKjl8hH/EsaVdnRaAEs0afY3VK8f+pCn0z/5n9xpiCd0UVrpoI3nwF1No4x/mEk71EW2ghH90hyg/9F0KHAHLneYZ6lCvcdhE/ENHrmc1nP0R0sUnOltV0XIyf0fEua7rvSqv0P0iuE0xD2+WUqh//96WFUP+XyeQxvn16XPBT8PSfE/nU/FQ; nx_ssl=2; page_uid=h1WRNsp0YiRssKvZbrNssssssMh-389234; BMR='
        }
# Send the request
        scrapy.http.Request(url, method='GET' , headers = headers,  dont_filter=False)

        print(response.body) #If the response is HTML
        #If the response is json ; import json
        #jsonresponse = json.loads(response.body_as_unicode())
        #print jsonresponse

In [20]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess(settings={
    "FEEDS": {
        "items.json": {"format": "json"},
    },
})

process.crawl(AaidSpider)
process.start() # the script will block here until the crawling is finished

2022-10-28 13:48:16 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-10-28 13:48:16 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.10.6 (main, Oct  7 2022, 20:19:58) [GCC 11.2.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 35.0.0, Platform Linux-4.15.0-143-generic-x86_64-with-glibc2.27
2022-10-28 13:48:16 [scrapy.crawler] INFO: Overridden settings:
{}
2022-10-28 13:48:16 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-28 13:48:16 [scrapy.extensions.telnet] INFO: Telnet Password: 55647e8cfd53ff6f
2022-10-28 13:48:16 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-10-28 13:48:16 [scrapy.middleware] INFO: Enabl

ReactorNotRestartable: 